# Calcul des dépenses estimées pour une assurance américaine selon le profil de l'assuré

## Données

Le dataset provient de ce [lien](https://www.kaggle.com/datasets/mirichoi0218/insurance) kaggle.

In [47]:
import pandas as pd

df=pd.read_csv('/content/insurance.csv')

In [48]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [49]:
#Quelques stats descriptives des variables numériques
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


Les données semblent très propres.

## Exploration et stats descriptives

Commençons par visualiser les données pour s'approprier le dataset.

In [51]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [52]:
# Création des pie charts avec px
fig1 = px.pie(df, names='smoker')
fig2 = px.pie(df, names='sex')
fig3 = px.pie(df, names='region')

fig = make_subplots(
    rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
    subplot_titles=("Répartition fumeur", "Répartition du sexe", "Répartition des régions")
)
# Ajout des pie charts avec labels intégrés directement
fig.add_trace(go.Pie(labels=fig1.data[0].labels, values=fig1.data[0].values,
                     textinfo='label+percent', insidetextorientation='auto'),
              row=1, col=1)

fig.add_trace(go.Pie(labels=fig2.data[0].labels, values=fig2.data[0].values,
                     textinfo='label+percent', insidetextorientation='auto'),
              row=1, col=2)

fig.add_trace(go.Pie(labels=fig3.data[0].labels, values=fig3.data[0].values,
                     textinfo='label+percent', insidetextorientation='auto'),
              row=1, col=3)

# Suppression de la légende globale
fig.update_layout(showlegend=False, height=500, width=1000, title_text="Trois répartitions", title_x=0.5)

fig.show()

On passe aux variables numeriques

In [53]:
# Créer 3 histogrammes avec px (pour différentes variables)
hist1 = px.histogram(df, x='children')
hist2 = px.histogram(df, x='bmi')
hist3 = px.histogram(df, x='age')

# Créer la figure avec 1 ligne, 3 colonnes
fig = make_subplots(rows=1, cols=3, subplot_titles=["Nombre d'enfants", "Répartition du bmi", "Repartition de l'age"])

# Ajouter les histogrammes dans chaque colonne
fig.add_trace(hist1.data[0], row=1, col=1)
fig.add_trace(hist2.data[0], row=1, col=2)
fig.add_trace(hist3.data[0], row=1, col=3)

# Mise en page
fig.update_layout(height=500, width=1000, title_text="Trois histogrammes", title_x=0.5)

fig.show()

In [62]:
#on affiche la charge en fonction de l'age
df_mean = df.groupby('age', as_index=False)['charges'].mean()
fig = px.line(df_mean, x='age', y='charges', title="Charge moyenne par âge")

# Étape 3 (optionnelle) : Mise en forme
fig.update_layout(xaxis_title='Âge', yaxis_title='Charge moyenne', title_x=0.5)

fig.show()

On observe par exemple que, en moyenne, la charge semble augmenter avec l'age

## Preprocessing


Pour préprocesser les données, on va :
- séparer nos données en échantillon train et test
- scaler toutes les données numériques
- mettre en colonne les données catégorielles
- entrainer les données train
- fiter les données test avec un modèle de regression et Random Forest
- évaluer les erreurs

### Split X et y

On cherche à prédire la charges (y) à partir des autres variables explicatives du dataset.

In [54]:
X=pd.DataFrame(df.drop(columns='charges'))
y=df['charges']

### Train-test split

On split nos données en train et test (80/20)!

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,y,random_state=0,test_size=0.2)

On met en place un preprocessor qui scale les variables numériques et qui detaille les valeurs des variables catégorielles

In [56]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# Préparation du préprocesseur
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), ['age','bmi','children']),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['sex','smoker','region'])
])

Pipeline avec preprocesseur et modèle de regression random forest

In [57]:
#On met en place la pipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor())
])

Application

In [58]:
#On fit sur le train et on test sur le test

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

In [64]:
#On evalue l'erreur MAE, R² et RMSE
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import numpy as np

mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE : {mae:.2f}")
print(f"R² : {r2:.3f}")
print(f"RMSE : {rmse:.2f}")

MAE : 2620.48
R² : 0.880
RMSE : 4373.35


Les variables du modèle expliquent à 88% la charge. Le modèle choisi semble donc robuste.
L'erreur moyenne est de 2620$.

On applique le modèle pour un nouveau client

In [60]:
nouveau_profil = pd.DataFrame([{
    'age': 27,
    'sex': 'male',
    'bmi': 27,
    'children': 0,
    'smoker' : 'no',
    'region' : 'southwest'
}])
prediction = pipeline.predict(nouveau_profil)
print(prediction)

[3703.5544047]


Pour un homme, de 27 ans, ayant un IMV de 27, sans enfant, non fumeur et habitant au sud ouest des etats unis, le montant de charges d'assurance estimé est de 3104 $.